<H1>IBM Applied Data Science Capstone Course by Coursera<H1>

<H1>PLACES FOR OPENING A NEW VEGAN RESTAURANT COMPLEX  AT MUMBAI, ECONOMIC HUB OF INDIA<H1>

**BY ARDHENDU MAHATHA**

**IMPORTING AND DOWNLOADING LIBRARIES**

In [1]:
import pandas as pd
import numpy as np
import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

In [2]:
from bs4 import BeautifulSoup # library to parse HTML and XML documents


In [12]:
! pip install geocoder

     |████████████████████████████████| 102kB 11.3MB/s ta 0:00:01


In [13]:
import geocoder

**TAKING DATA FROM WIKIPEDIA**

In [14]:
mumbai_dataset = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text

In [15]:
soup_mum = BeautifulSoup(mumbai_dataset, 'html.parser')

In [16]:
mumbai_neighbourhood= []

In [17]:
for row in soup_mum.find_all("div", class_="mw-category")[0].findAll("li"):
    mumbai_neighbourhood.append(row.text)

In [18]:
df_mum = pd.DataFrame({"Neighborhood": mumbai_neighbourhood})

df_mum.head()

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup


**FINDING GEOGRAPHICAL COORDINATES**

In [21]:
def latitude_longitude(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [22]:
coordinates = [ latitude_longitude(neighborhood) for neighborhood in df_mum["Neighborhood"].tolist() ]

In [23]:
df_coordinates = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])

In [24]:
df_coordinates.head()

,Latitude,Longitude
0,19.118459,72.841763
1,19.042830,72.927340
2,19.062940,72.926630
3,19.054370,72.840170
4,19.145560,72.948560


In [25]:
df_mum['Latitude'] = df_coordinates['Latitude']
df_mum['Longitude'] = df_coordinates['Longitude']

In [26]:
df_mum.head()

,Neighborhood,Latitude,Longitude
0,Andheri,19.118459,72.841763
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062940,72.926630
3,Bandra,19.054370,72.840170
4,Bhandup,19.145560,72.948560


In [27]:
df_mum.to_csv("mumbai.csv", index=False) #saving it

<h3>Plotting Map of Mumbai with its Neighbourhoods<h3>

In [28]:
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai,India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai,India 18.9387711, 72.8353355.


In [31]:
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df_mum['Latitude'], df_mum['Longitude'], df_mum['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mumbai)  
    
map_mumbai

In [32]:
map_mumbai.save("mapmumbai.html")

<h3>Using foursquare to explore neighbourhoods<h3>

In [33]:
CLIENT_ID = 'XQ4NB0KW4DT3CI5L1ADSQY4MIKJIKNPUCA3YBV4ZG2CIJD3N' # your Foursquare ID
CLIENT_SECRET = 'EKCRMQIWU4CADVS0MWTTR4D355SUKQ5OWQDSX1MPOZMHIM1C' # your Foursquare Secret
VERSION = '20200202'

In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()
        result2=pd.DataFrame(results)
        
        result1=result2["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in result1])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'VenueLatitude', 
                  'VenueLongitude', 
                  'VenueCategory']
    
    return(nearby_venues)

In [45]:
mumbai_venues = getNearbyVenues(names=df_mum['Neighborhood'],
                                latitudes=df_mum['Latitude'],
                                longitudes=df_mum['Longitude'],
                                )


Andheri
Anushakti Nagar
Baiganwadi
Bandra
Bhandup
Borivali
Charkop
Chembur
Dahisar
Devipada
Dombivli
Eastern Suburbs (Mumbai)
Ghatkopar
Goregaon
Grant Road
Jogeshwari
Juhu
Kalyan
Kandivali
Kanjurmarg
Kausa
Kurla
Mahavir Nagar (Kandivali)
Mankhurd
Matharpacady, Mumbai
Mira Road
Mogra Village
Mulund
Mumbra
Pestom sagar
Seven Bungalows
Shil Phata
Sion, Mumbai
Sonapur, Bhandup
Thakur village
Tilak Nagar (Mumbai)
Uttan
Vashi
Vikhroli
Wadala
Western Suburbs (Mumbai)
Worli


In [47]:
df_mumbai_venues=pd.DataFrame(mumbai_venues)
df_mumbai_venues.head()

,Neighborhood,Latitude,Longitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118459,72.841763,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118459,72.841763,Cafe Alfa,19.119667,72.843560,Indian Restaurant
2,Andheri,19.118459,72.841763,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
3,Andheri,19.118459,72.841763,McDonald's,19.119691,72.846102,Fast Food Restaurant
4,Andheri,19.118459,72.841763,Shoppers Stop,19.114935,72.842708,Department Store


<H1>Grouping the venues by neighbourhood<h1>

In [48]:
df_mumbai_venues.groupby(["Neighborhood"]).count()

,Latitude,Longitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,17,17,17,17,17,17
Anushakti Nagar,4,4,4,4,4,4
Baiganwadi,1,1,1,1,1,1
Bandra,12,12,12,12,12,12
Bhandup,2,2,2,2,2,2
Borivali,14,14,14,14,14,14
Charkop,4,4,4,4,4,4
Chembur,20,20,20,20,20,20
Dahisar,9,9,9,9,9,9


In [49]:
print('no of unique categories are {}.'.format(len(df_mumbai_venues['VenueCategory'].unique())))

no of unique categories are 105.


In [53]:
df_mumbai_venues.shape

(449, 7)

<h3>One hot encoding<h3>

In [51]:
onehot_mumbai = pd.get_dummies(df_mumbai_venues[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot_mumbai['Neighborhoods'] = df_mumbai_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot_mumbai.columns[-1]] + list(onehot_mumbai.columns[:-1])
onehot_mumbai = onehot_mumbai[fixed_columns]

print(onehot_mumbai.shape)
onehot_mumbai.head()

(449, 106)


,Neighborhoods,ATM,Accessories Store,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,...,Stadium,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Theme Park,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Women's Store
0,Andheri,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Andheri,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Andheri,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [52]:
mumbai_grouped = onehot_mumbai.groupby(["Neighborhoods"]).mean().reset_index()

print(mumbai_grouped.shape)
mumbai_grouped

(39, 106)


,Neighborhoods,ATM,Accessories Store,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,...,Stadium,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Theme Park,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Women's Store
0,Andheri,0.0,0.058824,0.000000,0.000000,0.000000,0.000000,0.058824,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.058824,0.00,0.000000
1,Anushakti Nagar,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000
2,Baiganwadi,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000
3,Bandra,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.083333,0.000000,0.00,0.000000
4,Bhandup,0.0,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.500000,0.000000,0.00,0.000000
5,Borivali,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000
6,Charkop,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000
7,Chembur,0.0,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.100000,0.00,0.000000
8,Dahisar,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.111111,0.000000,0.00,0.000000
9,Devipada,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000


In [59]:
len(mumbai_grouped[mumbai_grouped["Vegetarian / Vegan Restaurant"] > 0])

10

**_Data frame for Vegan Resturant_**

In [62]:
mumbai_vegan_restraunt= mumbai_grouped[["Neighborhoods","Vegetarian / Vegan Restaurant"]]
mumbai_vegan_restraunt.head()

,Neighborhoods,Vegetarian / Vegan Restaurant
0,Andheri,0.058824
1,Anushakti Nagar,0.000000
2,Baiganwadi,0.000000
3,Bandra,0.000000
4,Bhandup,0.000000


<h2>K-Means clustering of neighborhoods<h2>

In [70]:
no_of_clusters = 3

clustering_mumbai = mumbai_vegan_restraunt.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=no_of_clusters, random_state=0).fit(clustering_mumbai)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 2, 2, 2, 2, 2, 0, 2, 2], dtype=int32)

**ADDING CLUSTERING LABELS**

In [71]:
mumbai_merged = mumbai_vegan_restraunt.copy()

# add clustering labels
mumbai_merged["Cluster Labels"] = kmeans.labels_

In [72]:
mumbai_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mumbai_merged.head()

,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels
0,Andheri,0.058824,0
1,Anushakti Nagar,0.000000,2
2,Baiganwadi,0.000000,2
3,Bandra,0.000000,2
4,Bhandup,0.000000,2


**MERGING THE DATAFRAME WITH CLUSTER LABELS**

In [73]:
mumbai_merged = mumbai_merged.join(df_mum.set_index("Neighborhood"), on="Neighborhood")

print(mumbai_merged.shape)
mumbai_merged.head()

(39, 5)


,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
0,Andheri,0.058824,0,19.118459,72.841763
1,Anushakti Nagar,0.000000,2,19.042830,72.927340
2,Baiganwadi,0.000000,2,19.062940,72.926630
3,Bandra,0.000000,2,19.054370,72.840170
4,Bhandup,0.000000,2,19.145560,72.948560


In [74]:
mumbai_merged.sort_values(["Cluster Labels"], inplace=True)
mumbai_merged

,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
0,Andheri,0.058824,0,19.118459,72.841763
17,Kalyan,0.041667,0,18.953937,72.820367
16,Juhu,0.055556,0,19.014920,72.845220
34,Vashi,0.076923,0,19.084650,72.904810
25,Mira Road,0.142857,0,19.265674,72.870681
12,Ghatkopar,0.052632,0,19.086523,72.909008
20,Kausa,0.076923,0,19.127580,72.825390
7,Chembur,0.100000,0,19.062180,72.902410
30,"Sion, Mumbai",0.100000,0,19.043410,72.863320
21,Kurla,0.250000,1,19.064980,72.880690


**PLOTTING THE CLUSTERS**

In [77]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(no_of_clusters)
ys = [i+x+(i*x)**2 for i in range(no_of_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_merged['Latitude'], mumbai_merged['Longitude'], mumbai_merged['Neighborhood'], mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [78]:
map_clusters.save('mumbai_map_clusters.html')

<h1>Examining the clusters<h1>

<h3>cluster0<h3>

In [79]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 0]

,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
0,Andheri,0.058824,0,19.118459,72.841763
17,Kalyan,0.041667,0,18.953937,72.820367
16,Juhu,0.055556,0,19.014920,72.845220
34,Vashi,0.076923,0,19.084650,72.904810
25,Mira Road,0.142857,0,19.265674,72.870681
12,Ghatkopar,0.052632,0,19.086523,72.909008
20,Kausa,0.076923,0,19.127580,72.825390
7,Chembur,0.100000,0,19.062180,72.902410
30,"Sion, Mumbai",0.100000,0,19.043410,72.863320


<h3>Cluster1<h3>

In [80]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 1]

,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
21,Kurla,0.25,1,19.06498,72.88069


<h3>Cluster2<h3>

In [82]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 2]

,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
31,"Sonapur, Bhandup",0.0,2,19.163940,72.935440
27,Mumbra,0.0,2,19.167632,73.021408
32,Thakur village,0.0,2,19.210200,72.875410
26,Mulund,0.0,2,19.171830,72.955650
33,Tilak Nagar (Mumbai),0.0,2,18.996160,72.852790
24,"Matharpacady, Mumbai",0.0,2,18.950694,72.827268
35,Vikhroli,0.0,2,19.111090,72.927810
36,Wadala,0.0,2,19.017200,72.858170
23,Mankhurd,0.0,2,19.048530,72.932220
22,Mahavir Nagar (Kandivali),0.0,2,19.210940,72.841370


**Vegan restaurants are present in maximum nos at Kurla area. And it is in moderate amount at areas like Andheri, Juhu, Kalyan, Vashi, Chembur etc. and present in a very low amount at areas like Worli, Borivalli, Bhandup etc. So, businessmen wishing to open Vegan restaurants should not approach the Kurla area,they should focus on the areas of cluster 2 like Worli, Borivalli, Bhandup etc. These areas will have a zero competition and a total monopoly in the market can be achieved. Cluster 1 places can also be used for the business, since there is  a little competition and if the food quality of yours is better, then there is a very nice scope. And further it is advised to avoid the Kurla area as there is already a high concentration of Vegan Restaurants existing there.**